# Building a Neural Net From Scratch

## Processing Data

In [4]:
import pandas as pd
import numpy as np
from random import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('../data/titanic_data.csv')

In [5]:
data.drop(labels=['Cabin', 'Ticket', 'Name', 'PassengerId'], axis=1,inplace=True)
data.dropna(inplace=True)
features = pd.get_dummies(data, prefix=['Sex', 'Embarked'])

In [6]:
x = features.drop(labels=['Survived'], axis=1)
X = StandardScaler().fit_transform(x)
y = features.Survived.to_numpy().reshape(-1, 1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=322)

In [8]:
training_data = np.hstack((X_train, y_train))

## Neural Net Structure

In [9]:
def create_network_structure(n_input: int, n_layers: int, n_output: int) -> list:
    layers = [{'w' : [random() for i in range(n_input + 1)]} for i in range(n_layers)]
    output_layer = [{'w' : [random() for i in range(n_layers + 1)]} for i in range(n_output)]
    
    network = []
    network.append(layers)
    network.append(output_layer)
    
    return network

In [14]:
nn = create_network_structure(10, 2, 2)

In [11]:
def process_inputs(inputs: list, weights: list) -> float:
    products = [weights[i] * inputs[i] for i in range(len(weights) - 1)]
    processed = weights[-1] + sum(products)
    
    return processed

In [19]:
def activation_function(processed_input: float):
    activation = 1.0 / (1.0 + np.exp(-processed_input))
    
    return activation

In [20]:
def feed_forward(network: list, inputs: list) -> list:
    
    for layer in network:
        outputs = []
        for neuron in layer:
            neuron['Z'] = process_inputs(inputs, neuron['w'])
            neuron['output'] = activation_function(neuron['Z'])
            outputs.append(neuron['output'])
        inputs = outputs 
        
    return outputs

In [22]:
def activation_derivative(processed_input: float) -> float:
    return activation_function(processed_input) * (1 - activation_function(processed_input))

In [ ]:
inputs = training_data[0][:-1]

In [21]:
feed_forward(nn, inputs)

[0.789705081282761, 0.8105293712218309]

In [23]:
def propagate_errors(network: list, y: list) -> list:
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        
        if i == len(network) - 1:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(y[j] - neuron['output'])
        
        else:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[j+1]:
                    error += (neuron['w'][j] * neuron['error'])
                errors.append(error)
        
        for j in range(len(layer)):
            neuron['error'] = errors[j] * activation_derivative(neuron['output'])
    
    return network

In [66]:
class NeuralNet():
    
    def __init__(self, n_input: int, n_layers: int, n_output: int, learning_rate: float):
        self.n_input = n_input
        self.n_layers = n_layers
        self.n_output = n_output
        self.learning_rate = learning_rate
        
    def create_network_structure(self):
        layers = [{'w':[random() for i in range(self.n_input + 1)]} for i in range(self.n_layers)]
        output_layer = [{'w':[random() for i in range(self.n_layers + 1)]} 
                        for i in range(self.n_output)]
        network = []
        network.append(layers)
        network.append(output_layer)
        
        self.network = network
        
    def process_inputs(self, inputs: list, weights: list) -> float:
        products = [weights[i] * inputs[i] for i in range(len(weights) - 1)]
        processed = weights[-1] + sum(products)
    
        return processed
    
    def activation_function(self, processed: float) -> float:
    
        activation = 1.0 / (1.0 + np.exp(-processed))
        return activation
    
    def activation_derivative(self, processed_input: float) -> float:
    
        return self.activation_function(processed_input) * (1 - self.activation_function(processed_input))
    
    def feed_forward(self, inputs: list):
        
        for layer in self.network:
            outputs = []
            for neuron in layer:
                neuron['Z'] = self.process_inputs(inputs, neuron['w'])
                neuron['output'] = self.activation_function(neuron['Z'])
                outputs.append(neuron['output'])
            inputs = outputs
        
        return outputs

    def propagate_errors(self, y):
        for i in reversed(range(len(self.network))):
            layer = self.network[i]
            errors = []
            if i == len(self.network) - 1:
                # Final layer
                for j in range(len(layer)):
                    neuron = layer[j]
                    errors.append(y[j] - neuron['output'])
                # Hidden layers
            else:
                for j in range(len(layer)):
                    error = 0.0
                    for neuron in self.network[i + 1]:
                        error += (neuron['w'][j] * neuron['error'])
                    errors.append(error)
            for j in range(len(layer)):
                neuron = layer[j]
                neuron['error'] = errors[j] * self.activation_derivative(neuron['output'])
        
    def back_propagation(self, instance: list, y):
        self.propagate_errors(y)
        for i in range(len(self.network)):
            if i == 0:
                inputs = instance[:-1]
            else:
                inputs = [neuron['output'] for neuron in self.network[i - 1]]
            
            for neuron in self.network[i]:
                for j in range(len(inputs)):
                    neuron['w'][j] += self.learning_rate * neuron['error'] * inputs[j]
                neuron['w'][-1] += self.learning_rate * neuron['error']
    
    def train(self, training_data, n_epoch: int):
        for epoch in range(n_epoch):
            for instance in training_data:
                x = instance[:-1]
                y_pred = self.feed_forward(x)
                y_actual = [0 for i in range(self.n_output)]
                y_actual[int(instance[-1])] = 1
                
                self.back_propagation(x, y_actual)
            
    def predict(self, instance):
        output = self.feed_forward(instance)
        return output.index(max(output))

In [64]:
nn_steve = NeuralNet(training_data.shape[1] - 1, 2, 2, 0.4)
nn_steve.create_network_structure()

In [65]:
nn_steve.train(training_data, n_epoch = 5)

0.36629637289182504
0.3228873846046264
0.3218075633368748
0.3210882669209092
0.32060145709587934


In [62]:
for i in range(len(X_test)):
    prediction = nn_steve.predict(X_test[i])
    print('Expected label: ', y_test[i])
    print('Predicted label: ', prediction)

Expected label:  [0.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  0
Expected label:  [0.]
Predicted label:  0
Expected label:  [1.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  1
Expected label:  [1.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  1
Expected label:  [1.]
Predicted label:  0
Expected label:  [0.]
Predicted label:  0
Expected label:  [0.]
Predicted label:  0
Expected label:  [0.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  0
Expected label:  [1.]
Predicted label:  0
Expected label:  [0.]
Predicted label:  1
Expected label:  [1.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  0
Expected label:  [1.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  0
Expected label:  [0.]
Predicted label:  0
Expected label:  [0.]
Predicted label:  1
Expected label:  [0.]
Predicted label:  1
Expected label:  [1.]
Predicted label:  1
Expected label:  [1.]
Predicted la